## Setting up the notebook

In [1]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install sentencepiece

In [2]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [3]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
source_code = 'eng'
target_code = 'nso'

In [5]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing optok
#!git clone https://github.com/tatHi/optok4at.git
%cd optok4at/machineTranslation/optok
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/optok
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/optok
  Preparing metadata (setup.py) ... done
  Running setup.py develop for optok_nmt


In [6]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
%cd optok4at/machineTranslation/fairseq
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.0 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.9.0-0.editable-cp310-cp310-linux_x86_64.whl size=7879 sha256=ceaa609d581cf30ff36ac215941968068b6e341a11d43d403b55edb909cf1c2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-kmijl2p6/wheels/df/7c/ad/dad45b2aba84494676571009da0d5e5d4dc11b5a3eacbeb9fb
Successfully built fairseq


In [7]:
# installing youtokentome
!pip install youtokentome
import youtokentome as yttm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=1951574 sha256=89c4f944155482f5b79909e47904f522d7cf33a93803846164ff1338861bfde3
  Stored in directory: /root/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome


In [8]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# install multigram
#!git clone https://github.com/tatHi/multigram
%cd multigram
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/multigram
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/multigram
  Preparing metadata (setup.py) ... done
  Running setup.py develop for multigram


## Tokenizing the data

In [9]:
train_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{source_code}'
train_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/train.{target_code}'
val_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{source_code}'
val_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/val.{target_code}'
test_source_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{source_code}'
test_target_path = f'/content/drive/MyDrive/Research/eng-to-{target_code}/cleaned-data/test.{target_code}'

In [10]:
from multigram import lm
from multigram import tokenizer

In [11]:
mlm = lm.MultigramLM()
mlm.load('/content/drive/MyDrive/Research/eng-to-nso/target-tok/checkpoints-target-tok/checkpoint_best.optok.dec.mlm')
tknzr = tokenizer.Tokenizer(mlm)

In [12]:
# applying tokenization to training, validation and test sets
sp = spm.SentencePieceProcessor(model_file='/content/drive/MyDrive/Research/eng-to-nso/ulm/data/joint.model')

In [13]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok/data')

### Training set

In [14]:
with open(train_target_path, 'r') as tf:
  target = tf.read().splitlines()
  for i in range(len(target)):
    segment = target[i].split()
    for j in range(len(segment)):
      segment[j] = '▁' + segment[j]
    target[i] = ''.join(segment)

In [15]:
for i in range(len(target)):
  target[i] = tknzr.encode_as_pieces(target[i])

/content/drive/MyDrive/Research/eng-to-nso/multigram/multigram/lm.py:223: RuntimeWarning: divide by zero encountered in log
  logProbTable = np.log(probTable)


In [16]:
with open(f'train.{target_code}', 'w') as wf:
  for segment in target:
    wf.write(' '.join(segment) + '\n')

In [17]:
with open(train_source_path, 'r') as rf, open(f'train.{source_code}', 'w') as wf:
    for line in rf:
      wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

### Validation set

In [19]:
with open(val_target_path, 'r') as tf:
  target = tf.read().splitlines()
  for i in range(len(target)):
    segment = target[i].split()
    for j in range(len(segment)):
      segment[j] = '▁' + segment[j]
    target[i] = ''.join(segment)

In [20]:
for i in range(len(target)):
  target[i] = tknzr.encode_as_pieces(target[i])

/content/drive/MyDrive/Research/eng-to-nso/multigram/multigram/lm.py:223: RuntimeWarning: divide by zero encountered in log
  logProbTable = np.log(probTable)


In [21]:
with open(f'val.{target_code}', 'w') as wf:
  for segment in target:
    wf.write(' '.join(segment) + '\n')

In [22]:
with open(val_source_path, 'r') as rf, open(f'val.{source_code}', 'w') as wf:
    for line in rf:
      wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')

### Test set

In [23]:
with open(test_target_path, 'r') as tf:
  target = tf.read().splitlines()
  for i in range(len(target)):
    segment = target[i].split()
    for j in range(len(segment)):
      segment[j] = '▁' + segment[j]
    target[i] = ''.join(segment)

In [24]:
for i in range(len(target)):
  target[i] = tknzr.encode_as_pieces(target[i])

In [25]:
with open(f'test.{target_code}', 'w') as wf:
  for segment in target:
    wf.write(' '.join(segment) + '\n')

In [26]:
with open(test_source_path, 'r') as rf, open(f'test.{source_code}', 'w') as wf:
    for line in rf:
      wf.write(' '.join(sp.encode(line, out_type=str)) + '\n')